In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2cd7cff4c4dd4ac70f22fe5b9a12fd2f44feac07ec7ba4a47d7ab24494f1772d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from sentence_transformers import SentenceTransformer

train_file="/content/Qusetion.xlsx"
model=SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

df=pd.read_excel(train_file)
#embedding_vector열을 새롭게 생성
df['embedding_vector']=df['질문 (Query)'].progress_map(lambda x : model.encode(x))
#추가한 embedding_vector열이 있는 엑셀 파일 저장
#index=False : index열은 따로 저장X
df.to_excel('trian_Data_embedding.xlsx',index=False)

#torch.tensor()를 통해 텐서화 시킴
embedding_data=torch.tensor(df['embedding_vector'].tolist())
#embedding_data 파일명의 pt 파일로 저장 (pt파일은 쳇봇 엔진 구현에 사용)
torch.save(embedding_data,'embedding_data.pt')


100%|██████████| 326/326 [00:40<00:00,  8.01it/s]


In [ ]:
#임베딩 파일 로드와 유사도 측정
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

model=SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embedding_data=torch.load('/content/embedding_data.pt')
df=pd.read_excel('/content/trian_Data_embedding.xlsx')

#질문 예시문장
sentence='추천 시스템이 궁금해'

sentence=sentence.replace(" ","") #공백 제거


#질문 예시 문장 인코딩 후 텐서화
sentence_encode=model.encode(sentence) #인코딩
sentence_tensor=torch.tensor(sentence_encode) #텐서화

#저장한 임베딩 데이터와의 코사인 유사도 측정
cos_sim=util.cos_sim(sentence_tensor,embedding_data)


#선택된 질문 출력
best_sim_idx=int(np.argmax(cos_sim))
selected_qes=df['질문 (Query)'][best_sim_idx]


#선택된 질문 문장에 대한 인코딩
selected_qes_encode=model.encode(selected_qes)

#유사도 점수 측정
score=np.dot(sentence_tensor, selected_qes_encode)/(np.linalg.norm(sentence_tensor)*np.linalg.norm(selected_qes_encode))
print(f'선택된 질문과의 유사도 = {score}')

#답변
answer=df['답변 (Answer)'][best_sim_idx]
answer

선택된 질문과의 유사도 = 0.5919371247291565


'ai/ml 부서의 배지윤, 오윤선, 정다혜에게 물어보시면 됩니다. '